In [29]:
INPUTS_PATH = "/viscam/projects/BlueHalo/data/bluehalo_splits/wriva-datasets/wriva_inputs"
POSES_PATH = "/viscam/projects/BlueHalo/data/bluehalo_splits/wriva-datasets/wriva-output"

In [63]:
!ls {INPUTS_PATH}/*/input_images

/viscam/projects/BlueHalo/data/bluehalo_splits/wriva-datasets/wriva_inputs/0023_siteA10_single_camera_ground_100/input_images:
image_000001.jpg  image_000026.jpg  image_000051.jpg  image_000076.jpg
image_000002.jpg  image_000027.jpg  image_000052.jpg  image_000077.jpg
image_000003.jpg  image_000028.jpg  image_000053.jpg  image_000078.jpg
image_000004.jpg  image_000029.jpg  image_000054.jpg  image_000079.jpg
image_000005.jpg  image_000030.jpg  image_000055.jpg  image_000080.jpg
image_000006.jpg  image_000031.jpg  image_000056.jpg  image_000081.jpg
image_000007.jpg  image_000032.jpg  image_000057.jpg  image_000082.jpg
image_000008.jpg  image_000033.jpg  image_000058.jpg  image_000083.jpg
image_000009.jpg  image_000034.jpg  image_000059.jpg  image_000084.jpg
image_000010.jpg  image_000035.jpg  image_000060.jpg  image_000085.jpg
image_000011.jpg  image_000036.jpg  image_000061.jpg  image_000086.jpg
image_000012.jpg  image_000037.jpg  image_000062.jpg  image_000087.jpg
image_000013.jpg  ima

In [5]:
!ls {INPUTS_PATH}/

0023_siteA10_single_camera_ground_100  0_Origin		    1-3_ImageDensity_20
0024_siteA10_single_camera_ground_75   1-0_ImageDensity_40  1-4_ImageDensity_15
0025_siteA10_single_camera_ground_50   1-1_ImageDensity_30  1-5_ImageDensity_10
0026_siteA10_single_camera_ground_30   1-2_ImageDensity_25


In [45]:
!ls {POSES_PATH}/bh_colmap_output/wriva_inputs

0033  0034  0035  0036	0037  0038  0039


In [8]:
!ls {INPUTS_PATH}/1-0_ImageDensity_40

input_images  reference_images	  ta2_reference_images
manifest.csv  reference_metadata  ta2_reference_metadata


In [127]:
import os
import json
import glob

DATA_ROOT = "/viscam/projects/BlueHalo/data/wriva/siteA01-apl-office-buildings/"

def get_poses_json(poses_uid):
    path = os.path.join(POSES_PATH, f"bh_colmap_output/wriva_inputs/{poses_uid}/transforms.json")
    with open(path, 'r') as fp:
        return json.load(fp)

def get_image_path(file_name):
    _, site_idx, *date, basename = file_name.split('-')
    date = '-'.join(date)
    
    image_folder_pattern = os.path.join(DATA_ROOT, f"{site_idx}*")
    [image_folder] = glob.glob(image_folder_pattern)
    
    dated_image_folder = os.path.join(image_folder, date)
    assert os.path.exists(dated_image_folder), dated_image_folder
    
    image_path_pattern = os.path.join(dated_image_folder, f"*{basename}")
    
    [image_path] = glob.glob(image_path_pattern)
    return image_path

def get_all_scenes():
    scenes = []
    for uid in ['0033', '0034', '0035', '0036', '0037', '0038', '0039',]:
        poses_json = get_poses_json(uid)
        frames = poses_json['frames']
        file_paths = [frame['file_path'] for frame in frames]
        file_names = [os.path.basename(file_path) for file_path in file_paths]
        scene = []
        for file_name, frame in zip(file_names, frames):
            if 'site' in file_name:
                image_path = get_image_path(file_name)
                
                scene.append((image_path, frame))
        scenes.append(scene)
    return scenes
        
def central_crop_img_arr(img):
    h, w, c = img.shape
    assert min(h, w) == 256
    s = 256
    oh_resid = (h - s) % 2
    ow_resid = (w - s) % 2
    oh = (h - s) // 2
    ow = (w - s) // 2
    img = img[oh : h - oh - oh_resid, ow : w - ow - ow_resid]
    assert img.shape == (256, 256, c), img.shape
    return img

def fov_to_intrinsics_matrix(fov_deg, height):
    # assuming a square camera and fov in degrees and image with "height" pixels,
    # get the intrinsics matrix in our canonical format
    fov = np.deg2rad(fov_deg)
    focal = height / (2 * np.tan(fov / 2))
    pixtocam = get_pixtocam(focal, height, height)

    # homogenize it
    eye = np.eye(4)
    eye[:3, :3] = pixtocam
    return eye

def get_pixtocam(focal: float, width: float, height: float):
    """Inverse intrinsic matrix for a perfect pinhole camera."""
    camtopix = intrinsic_matrix(focal, focal, width * 0.5, height * 0.5)
    return np.linalg.inv(camtopix)

def intrinsic_matrix(fx: float, fy: float, cx: float, cy: float):
    """Intrinsic matrix for a pinhole camera in OpenCV coordinate system."""
    return np.array(
        [
            [fx, 0, cx],
            [0, fy, cy],
            [0, 0, 1.0],
        ]
    )

scenes = get_all_scenes()    

In [158]:
from PIL import Image
import numpy as np

def load_and_resize_image(image_path):
    with open(image_path, 'rb') as fp:
        image = Image.open(fp).convert("RGB")
    h,w = image.size
    s = min(h,w)
    r = 256/s
    image = image.resize((int(h*r), int(w*r)))
    assert min(image.size) == 256, image.size
    image_arr = np.array(image)
    return central_crop_img_arr(image_arr)

def convert_view(view):
    image_path, frame = view
    image = load_and_resize_image(image_path)
    fov_deg = min(frame['fovx'], frame['fovy'])
    
    cam2world = np.array(frame['transform_matrix'])
    intrinsics = fov_to_intrinsics_matrix(fov_deg, 256)
    return image, intrinsics, cam2world


def write_scenes(scenes, base_path):
    view_count = 0
    for scene_idx, scene in enumerate((scenes)):
        print(scene_idx)
        
        scene_dir = os.path.join(base_path, f'out{scene_idx}')
        os.makedirs(scene_dir, exist_ok=True)
        
        rgb_dir = os.path.join(scene_dir, 'rgb')
        os.makedirs(rgb_dir, exist_ok=True)
        
        pose_dir = os.path.join(scene_dir, 'pose')
        os.makedirs(pose_dir, exist_ok=True)
        
        for view_idx, view in enumerate(scene):
            image, intrinsics, extrinsics = convert_view(view)
            
            uid = "%.4d_sc%.4d_az%.2d" % (view_count, scene_idx, view_idx)
            
            # save image
            image_path = os.path.join(rgb_dir, f"{uid}.png")
            Image.fromarray(image).save(image_path)
            
            # save extrinsics
            pose_path = os.path.join(pose_dir, f"{uid}_RT.txt")
            extrinsics_lines = np.round(extrinsics, 4).tolist()
            extrinsics_lines = [' '.join(map(str, extrinsics_line)) for extrinsics_line in extrinsics_lines]
            extrinsics_str = '\n'.join(extrinsics_lines)
            with open(pose_path, 'w') as fp:
                fp.write(extrinsics_str)
                
            # save intrinsics
            if view_idx == 0:
                # assume constant intrinsics
                intrinsics_path = os.path.join(scene_dir, "intrinsics.txt")
                fx = np.linalg.inv(intrinsics)[0,0]
                fy = fx  # assume square image
                cx = 128
                cy = 128
                intrinsics_str = f"{fx} {fy} {cx} {cy}\n0 0 0\n1\n256 256\n"
                with open(intrinsics_path, 'w') as fp:
                    fp.write(intrinsics_str)
            
            view_count += 1
        
        # need to write pose
        
write_scenes(scenes, '/viscam/u/ksarge/COLF/wriva_train')

0
1
2
3
4
5
6


In [157]:
!ls /viscam/u/ksarge/COLF

config.py		     geometry.py	    room_diverse_train.zip
conv2d_gradfix.py	     hyperlayers.py	    summaries.py
convert_wriva_to_colf.ipynb  logs		    test_configs
conv_modules.py		     loss_functions.py	    test.py
custom_layers.py	     models.py		    torchmeta
dataio.py		     __pycache__	    train_configs
data_util.py		     README.md		    train.py
diff_operators.py	     room_diverse_test	    util.py
environment_viscam.yml	     room_diverse_test.zip
environment.yml		     room_diverse_train


In [153]:
fx = 280.0
fy = 373.3
cx = 128.0
cy = 128.0

print(intrinsics)

280.0 373.3 128.0 128.0
0 0 0
1
256 256

